<a href="https://colab.research.google.com/github/sahasubhajit/Tree-Graph-Search-Algorithms/blob/main/tree_searching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from min_heap import min_heap_push, min_heap_pop

# Denote the edges by adjacency matrix

In [2]:
def create_tree(numner_of_nodes, edges):
  adjancency_matrix = np.zeros([numner_of_nodes, numner_of_nodes])
  #print(adjancency_matrix[0,0])
  for edge in edges:
    #print(edge, adjancency_matrix[edge[0], edge[1]])
    adjancency_matrix[edge[0], edge[1]] = 1
    adjancency_matrix[edge[1], edge[0]] = 1
  return adjancency_matrix

#Algorithm of the search

In [3]:
class SEARCH:

  def __init__(self, tree, values):
    self.tree = tree
    self.paths = [[0]]
    self.node_values = values


  def add_path(self, node_index):
    for upper_node_index in range(node_index):
      #print(upper_node_index, self.tree[int(self.front)][upper_node_index])
      if int(self.tree[int(node_index)][upper_node_index]) == 1:
        for existing_path in self.paths:
            if existing_path[len(existing_path) - 1] == upper_node_index:
              self.new_path = existing_path.copy()
              self.new_path.append(node_index)
              self.paths.append(self.new_path)
              #print("After adding Path ", self.paths)


  def delete_path(self, node_index):
    for self.existing_path in self.paths:
      if self.existing_path[len(self.existing_path) - 1] == node_index:
        self.paths.remove(self.existing_path)
        #print("After deleting Path ", self.paths)





class BFS(SEARCH):

  def __init__(self, tree, values):
    super().__init__(tree, values)

  def enque(self, queue, node):
    #print("From function ", queue)
    self.queue_new = np.zeros(len(queue) + 1)
    self.queue_new[:len(queue)] = queue
    self.queue_new[len(queue)] = node
    del queue
    return self.queue_new


  def denque(self, queue):
    self.queue_new = queue[1:len(queue)]
    self.front = queue[0]
    del queue
    return self.queue_new, int(self.front)



  def run_bfs(self, root, target):
    self.queue =  np.array([root])
    self.depth = 0
    flag = 0
    if self.node_values[root] == target:
      return self.paths[0]
    else:
      while len(self.queue):
        #print("Queue ", self.queue)
        #print("Paths ", self.paths)
        self.queue, self.front = self.denque(self.queue)
        print("After DnQueue ", self.queue)
        #print("denque ", self.queue)
        for node in range(self.tree.shape[0]):
          if (self.tree[int(self.front)][node] == 1 and node > self.front) == True:
            print("Front: ", self.front, "Node: ", node)
            self.add_path(node)
            self.queue = self.enque(self.queue, node)
            print("After EnQueue ", self.queue)
            if target == self.node_values[node]:
              print("Goal Reached", target, node)
              self.queue = []
              flag = 1
              break
        self.delete_path(self.front)
      if flag == 0:
        print("No solution Exist! check the input node values")
        return -1
      else:
        return self.paths[len(self.paths)-1]






class DFS(SEARCH):

  def __init__(self, tree, values):
    super().__init__(tree, values)
    self.stack =  np.array([0])
    self.closed_list = []
    self.flag = 0
    self.depth = 0

  def push(self, stack, node):
    #print("From function ", queue)
    self.stack_new = np.zeros(len(stack) + 1)
    self.stack_new[:len(stack)] = stack
    self.stack_new[len(stack)] = node
    #print("push! new {} previous {}".format(self.stack_new, self.stack))
    del stack
    return self.stack_new


  def pop(self, stack):
    self.stack_new = stack[0:len(stack)-1]
    self.rear = stack[len(stack)-2]
    del stack
    #print("pop! new {} previous {}".format(self.stack_new, self.stack))
    return self.stack_new, self.rear


  def run_dfs(self, root, target, max_depth = 99999):
    self.max_depth = max_depth
    self.rear = root
    #self.depth = 0
    #self.flag = 0
    print("closed list {} and stack {} and depth {}".format(self.closed_list, self.stack, self.max_depth))
    if self.node_values[self.rear] == target:
      self.flag = 1
      self.stack = []
      return self.paths[len(self.paths)-1]
    else:
      while len(self.stack):
        temp = 0
        for node in range(self.tree.shape[0]):
          if (self.tree[int(self.rear)][node] == 1 and node > self.rear and node not in self.closed_list and self.depth < self.max_depth):
            temp += 1
            self.depth += 1
            #print("temp: ", temp, "Rear: ", self.rear, "Node: ", node)
            self.add_path(node)
            self.stack = self.push(self.stack, node)
            self.run_dfs(node, target, self.max_depth)
        if temp ==0:
            self.closed_list.append(self.rear)
            self.stack, self.rear = self.pop(self.stack)
            self.depth -= 1
            self.run_dfs(int(self.rear), target, self.max_depth)


    if self.flag == 0:
      print("No solution Exist! check the input node values")
      return -1
    else:
      return self.paths[len(self.paths)-1]



  def run_iterative_dfs(self, root, target):
    output = -1
    count = 0
    self.max_depth = 0
    while output == -1:
      self.stack =  np.array([0])
      self.closed_list = []
      self.flag = 0
      self.depth = 0
      output = self.run_dfs(root, target, self.max_depth)
      self.max_depth += 1
      print(self.max_depth, output)
      count += 1
      if len(self.closed_list) == len(self.node_values):
        output = -1
        break
    return output


class UNIFORM_COST_SEARCH(SEARCH):
    def __init__(self, tree, values, graph, city_as_keys, order_as_keys, straight_line_distances = False, add_heuristic = False):
      super().__init__(tree, values)
      self.priority_queue =  [{'Arad':0}] #initial path cost
      self.closed_list = []
      self.flag = 0
      self.depth = 0
      self.paths = [[0]]
      self.GRAPH = graph
      self.city_as_keys = city_as_keys
      self.order_as_keys = order_as_keys
      self.previously_visited_nodes = []
      self.add_huristic = add_heuristic
      self.straight_line_distances = straight_line_distances


    def search_neighbours_with_cost(self, node_index, node_cost):  #add neighbours of the poped node
      node_name = self.order_as_keys[str(node_index)]
      temp_subgraph = self.GRAPH[node_name]
      neighbour_names = list(temp_subgraph.keys())
      for neighbour in range(len(neighbour_names)):
         if neighbour_names[neighbour] not in self.previously_visited_nodes: #check if the neighbour has been traversed earlier
          temp_subgraph[neighbour_names[neighbour]] += node_cost       #node cost = previous node cost + last edge cost
          if self.add_huristic == True:
              temp_subgraph[neighbour_names[neighbour]] += self.straight_line_distances[neighbour_names[neighbour]]
         else:
          del temp_subgraph[neighbour_names[neighbour]]
      return temp_subgraph


    def heap_push(self, node):
      self.priority_queue  = min_heap_push(self.priority_queue, node)
      return self.priority_queue

    def heap_pop(self):
      return min_heap_pop(self.priority_queue)


    def run_ucs(self, target, root_index = 0):
      if self.order_as_keys[str(root_index)] == target:
        return [], 0
      else:
        present_node_index = root_index
        present_node_cost = 0
        while True:
          self.priority_queue, poped = self.heap_pop()             #pop from the min heap and then add it to the previously visited node
          self.previously_visited_nodes.append(list(poped)[0])
          self.add_path(int(self.city_as_keys[list(poped)[0]]))
          present_node_index =  self.city_as_keys[list(poped)[0]]
          present_node_cost = list(poped.values())[0]
          if self.order_as_keys[str(present_node_index)] == target: #If the poped node is the target node then stop
            break
          new_paths = self.search_neighbours_with_cost(present_node_index, present_node_cost)
          print("new_paths searched")
          for i in range(len(new_paths.values())):
            self.new_paths = []
            for key, value in new_paths.items():
              self.new_paths.append({str(key):value})
            self.priority_queue = self.heap_push(self.new_paths[i])
          print("pushed {} poped {} and previously_visited_nodes {}".format(self.priority_queue, poped, self.previously_visited_nodes))

        optimal_path = [self.order_as_keys[str(i)] for i in self.paths[len(self.paths)-1]]



        self.reset()

        return self.priority_queue, poped, optimal_path



    def reset(self):
      self.priority_queue =  [{'Arad':0}] #initial path cost
      self.closed_list = []
      self.flag = 0
      self.depth = 0
      self.paths = [[0]]
      self.city_as_keys = city_as_keys
      self.order_as_keys = order_as_keys
      self.previously_visited_nodes = []


In [4]:
GRAPH = {
            'Arad': {'Sibiu': 140, 'Zerind': 75, 'Timisoara': 118},
            'Zerind': {'Arad': 75, 'Oradea': 71},
            'Oradea': {'Zerind': 71, 'Sibiu': 151},
            'Sibiu': {'Arad': 140, 'Oradea': 151, 'Fagaras': 99, 'Rimnicu': 80},
            'Timisoara': {'Arad': 118, 'Lugoj': 111},
            'Lugoj': {'Timisoara': 111, 'Mehadia': 70},
            'Mehadia': {'Lugoj': 70, 'Drobeta': 75},
            'Drobeta': {'Mehadia': 75, 'Craiova': 120},
            'Craiova': {'Drobeta': 120, 'Rimnicu': 146, 'Pitesti': 138},
            'Rimnicu': {'Sibiu': 80, 'Craiova': 146, 'Pitesti': 97},
            'Fagaras': {'Sibiu': 99, 'Bucharest': 211},
            'Pitesti': {'Rimnicu': 97, 'Craiova': 138, 'Bucharest': 101},
            'Bucharest': {'Fagaras': 211, 'Pitesti': 101, 'Giurgiu': 90, 'Urziceni': 85},
            'Giurgiu': {'Bucharest': 90},
            'Urziceni': {'Bucharest': 85, 'Vaslui': 142, 'Hirsova': 98},
            'Hirsova': {'Urziceni': 98, 'Eforie': 86},
            'Eforie': {'Hirsova': 86},
            'Vaslui': {'Iasi': 92, 'Urziceni': 142},
            'Iasi': {'Vaslui': 92, 'Neamt': 87},
            'Neamt': {'Iasi': 87}
        }

order_as_keys = {
    '0': 'Arad',
    '1': 'Zerind',
    '2': 'Oradea',
    '3': 'Sibiu',
    '4': 'Timisoara',
    '5': 'Lugoj',
    '6': 'Mehadia',
    '7': 'Drobeta',
    '8': 'Craiova',
    '9': 'Rimnicu',
    '10': 'Fagaras',
    '11': 'Pitesti',
    '12': 'Bucharest',
    '13': 'Giurgiu',
    '14': 'Urziceni',
    '15': 'Hirsova',
    '16': 'Eforie',
    '17': 'Vaslui',
    '18': 'Iasi',
    '19': 'Neamt'
}

city_as_keys = {
    'Arad': '0',
    'Zerind': '1',
    'Oradea': '2',
    'Sibiu': '3',
    'Timisoara': '4',
    'Lugoj': '5',
    'Mehadia': '6',
    'Drobeta': '7',
    'Craiova': '8',
    'Rimnicu': '9',
    'Fagaras': '10',
    'Pitesti': '11',
    'Bucharest': '12',
    'Giurgiu': '13',
    'Urziceni': '14',
    'Hirsova': '15',
    'Eforie': '16',
    'Vaslui': '17',
    'Iasi': '18',
    'Neamt': '19'
}





# HERE THE STRAIGHT LINE DISTANCE VALUES ARE IN REFERENCE TO BUCHAREST AS THE DESTINATION
straight_line = {
    'Arad': 366,
    'Zerind': 374,
    'Oradea': 380,
    'Sibiu': 253,
    'Timisoara': 329,
    'Lugoj': 244,
    'Mehadia': 241,
    'Drobeta': 242,
    'Craiova': 160,
    'Rimnicu': 193,
    'Fagaras': 176,
    'Pitesti': 100,
    'Bucharest': 0,
    'Giurgiu': 77,
    'Urziceni':  80,
    'Hirsova': 151,
    'Eforie': 161,
    'Vaslui': 199,
    'Iasi': 226,
    'Neamt': 234
}


In [5]:
Values = []

for key in order_as_keys.keys():
  Values.append(key)


In [6]:
number_of_nodes = len(straight_line)
edges = []
for i in range(len(order_as_keys)):
  city_1 = order_as_keys[str(i)]
  for key in GRAPH[city_1]:
    for j in range(len(order_as_keys)):
      if order_as_keys[str(j)] == key:
        edges.append([i,j])

In [7]:
Tree_edges = create_tree(number_of_nodes, edges)

In [8]:
ucs = UNIFORM_COST_SEARCH(Tree_edges, Values, GRAPH, city_as_keys, order_as_keys)
queue, poped, optimal_path = ucs.run_ucs('Bucharest')

new_paths searched
pushed [{'Zerind': 75}, {'Timisoara': 118}, {'Sibiu': 140}] poped {'Arad': 0} and previously_visited_nodes ['Arad']
new_paths searched
pushed [{'Timisoara': 118}, {'Sibiu': 140}, {'Oradea': 146}] poped {'Zerind': 75} and previously_visited_nodes ['Arad', 'Zerind']
new_paths searched
pushed [{'Sibiu': 140}, {'Oradea': 146}, {'Lugoj': 229}] poped {'Timisoara': 118} and previously_visited_nodes ['Arad', 'Zerind', 'Timisoara']
new_paths searched
pushed [{'Oradea': 146}, {'Rimnicu': 220}, {'Lugoj': 229}, {'Fagaras': 239}, {'Oradea': 291}] poped {'Sibiu': 140} and previously_visited_nodes ['Arad', 'Zerind', 'Timisoara', 'Sibiu']
new_paths searched
pushed [{'Rimnicu': 220}, {'Fagaras': 239}, {'Lugoj': 229}, {'Oradea': 291}] poped {'Oradea': 146} and previously_visited_nodes ['Arad', 'Zerind', 'Timisoara', 'Sibiu', 'Oradea']
new_paths searched
pushed [{'Lugoj': 229}, {'Fagaras': 239}, {'Oradea': 291}, {'Craiova': 366}, {'Pitesti': 317}] poped {'Rimnicu': 220} and previously_

In [9]:
optimal_path

['Arad', 'Sibiu', 'Rimnicu', 'Pitesti', 'Bucharest']

In [10]:
poped

{'Bucharest': 418}

In [11]:
ucs = UNIFORM_COST_SEARCH(Tree_edges, Values, GRAPH, city_as_keys, order_as_keys, straight_line, True)
queue, poped, optimal_path = ucs.run_ucs('Bucharest')

new_paths searched
pushed [{'Sibiu': 393}, {'Timisoara': 447}, {'Zerind': 449}] poped {'Arad': 0} and previously_visited_nodes ['Arad']
new_paths searched
pushed [{'Timisoara': 447}, {'Zerind': 449}, {'Rimnicu': 806}, {'Fagaras': 808}, {'Oradea': 1064}] poped {'Sibiu': 393} and previously_visited_nodes ['Arad', 'Sibiu']
new_paths searched
pushed [{'Zerind': 449}, {'Fagaras': 808}, {'Rimnicu': 806}, {'Oradea': 1064}, {'Lugoj': 920}] poped {'Timisoara': 447} and previously_visited_nodes ['Arad', 'Sibiu', 'Timisoara']
new_paths searched
pushed [{'Rimnicu': 806}, {'Fagaras': 808}, {'Lugoj': 920}, {'Oradea': 1064}, {'Oradea': 975}] poped {'Zerind': 449} and previously_visited_nodes ['Arad', 'Sibiu', 'Timisoara', 'Zerind']
new_paths searched
pushed [{'Fagaras': 808}, {'Oradea': 975}, {'Lugoj': 920}, {'Oradea': 1064}, {'Craiova': 1332}, {'Pitesti': 1223}] poped {'Rimnicu': 806} and previously_visited_nodes ['Arad', 'Sibiu', 'Timisoara', 'Zerind', 'Rimnicu']
new_paths searched
pushed [{'Lugoj'

In [12]:
optimal_path

['Arad', 'Sibiu', 'Rimnicu', 'Pitesti', 'Bucharest']

In [13]:
poped

{'Bucharest': 1258}

# TestCase 1: Small complete binary tree

In [14]:
Tree_edges = create_tree(7, [[0,1], [2,0], [1,3], [1,4], [2,5], [2,6]])
fitted_tree = BFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17])
optimal_path = fitted_tree.run_bfs(0, -3)
print("The Optimal Path index by BFS", optimal_path)


fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17])
optimal_path = fitted_tree.run_dfs(0, -3)
print("The Optimal Path index by DFS", optimal_path)



fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17])
optimal_path = fitted_tree.run_dfs(0, -3, 1)
print("The Optimal Path index by Depth limited Search", optimal_path)



fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17])
optimal_path = fitted_tree.run_iterative_dfs(0, -3)
print("The Optimal Path index by iterative DFS", optimal_path)

After DnQueue  []
Front:  0 Node:  1
After EnQueue  [1.]
Front:  0 Node:  2
After EnQueue  [1. 2.]
After DnQueue  [2.]
Front:  1 Node:  3
After EnQueue  [2. 3.]
Front:  1 Node:  4
After EnQueue  [2. 3. 4.]
After DnQueue  [3. 4.]
Front:  2 Node:  5
After EnQueue  [3. 4. 5.]
Goal Reached -3 5
The Optimal Path index by BFS [0, 2, 5]
closed list [] and stack [0] and depth 99999
closed list [] and stack [0. 1.] and depth 99999
closed list [] and stack [0. 1. 3.] and depth 99999
closed list [3] and stack [0. 1.] and depth 99999
closed list [3] and stack [0. 1. 4.] and depth 99999
closed list [3, 4] and stack [0. 1.] and depth 99999
closed list [3, 4, 1] and stack [0.] and depth 99999
closed list [3, 4, 1] and stack [0. 2.] and depth 99999
closed list [3, 4, 1] and stack [0. 2. 5.] and depth 99999
The Optimal Path index by DFS [0, 2, 5]
closed list [] and stack [0] and depth 1
closed list [] and stack [0. 1.] and depth 1
closed list [1] and stack [0.] and depth 1
closed list [1] and stack [0.

# TestCase 2: Check on non complete tree

In [15]:
Tree_edges = create_tree(7, [[0,1], [2,0], [1,3], [1,4], [1,5], [2,6]])
fitted_tree = BFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17])
optimal_path = fitted_tree.run_bfs(0, 17)
print("The Optimal Path index by BFS", optimal_path)


fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17])
optimal_path = fitted_tree.run_dfs(0, 17)
print("The Optimal Path index by DFS", optimal_path)



fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17])
optimal_path = fitted_tree.run_dfs(0, 17, 6)
print("The Optimal Path index by Depth Limited Search", optimal_path)



fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17])
optimal_path = fitted_tree.run_iterative_dfs(0, 17)
print("The Optimal Path index by iterative DFS", optimal_path)

After DnQueue  []
Front:  0 Node:  1
After EnQueue  [1.]
Front:  0 Node:  2
After EnQueue  [1. 2.]
After DnQueue  [2.]
Front:  1 Node:  3
After EnQueue  [2. 3.]
Goal Reached 17 3
The Optimal Path index by BFS [0, 1, 3]
closed list [] and stack [0] and depth 99999
closed list [] and stack [0. 1.] and depth 99999
closed list [] and stack [0. 1. 3.] and depth 99999
The Optimal Path index by DFS [0, 1, 3]
closed list [] and stack [0] and depth 6
closed list [] and stack [0. 1.] and depth 6
closed list [] and stack [0. 1. 3.] and depth 6
The Optimal Path index by Depth Limited Search [0, 1, 3]
closed list [] and stack [0] and depth 0
closed list [0] and stack [] and depth 0
No solution Exist! check the input node values
No solution Exist! check the input node values
1 -1
closed list [] and stack [0] and depth 1
closed list [] and stack [0. 1.] and depth 1
closed list [1] and stack [0.] and depth 1
closed list [1] and stack [0. 2.] and depth 1
closed list [1, 2] and stack [0.] and depth 1
cl

# TestCase 3: Check on bigger Tree

In [16]:
Tree_edges = create_tree(9, [[0,1], [2,0], [1,3], [1,4], [1,5], [2,6], [3,7], [3,8]])
fitted_tree = BFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17, 15, -1])
optimal_path = fitted_tree.run_bfs(0, 15)
print("The Optimal Path index by BFS ", optimal_path)




fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17, 15, -1])
optimal_path = fitted_tree.run_dfs(0, 15)
print("The Optimal Path index by DFS ", optimal_path)



fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17, 15, -1])
optimal_path = fitted_tree.run_dfs(0, 15, 2)
print("The Optimal Path index by Depth Limited Search ", optimal_path)



fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17, 15, -1])
optimal_path = fitted_tree.run_iterative_dfs(0, 15)
print("The Optimal Path index by iterative DFS ", optimal_path)

After DnQueue  []
Front:  0 Node:  1
After EnQueue  [1.]
Front:  0 Node:  2
After EnQueue  [1. 2.]
After DnQueue  [2.]
Front:  1 Node:  3
After EnQueue  [2. 3.]
Front:  1 Node:  4
After EnQueue  [2. 3. 4.]
Front:  1 Node:  5
After EnQueue  [2. 3. 4. 5.]
After DnQueue  [3. 4. 5.]
Front:  2 Node:  6
After EnQueue  [3. 4. 5. 6.]
After DnQueue  [4. 5. 6.]
Front:  3 Node:  7
After EnQueue  [4. 5. 6. 7.]
Goal Reached 15 7
The Optimal Path index by BFS  [0, 1, 3, 7]
closed list [] and stack [0] and depth 99999
closed list [] and stack [0. 1.] and depth 99999
closed list [] and stack [0. 1. 3.] and depth 99999
closed list [] and stack [0. 1. 3. 7.] and depth 99999
The Optimal Path index by DFS  [0, 1, 3, 7]
closed list [] and stack [0] and depth 2
closed list [] and stack [0. 1.] and depth 2
closed list [] and stack [0. 1. 3.] and depth 2
closed list [3] and stack [0. 1.] and depth 2
closed list [3] and stack [0. 1. 4.] and depth 2
closed list [3, 4] and stack [0. 1.] and depth 2
closed list [

# TestCase 4: Solution is at the root of the tree

In [17]:
Tree_edges = create_tree(9, [[0,1], [2,0], [1,3], [1,4], [1,5], [2,6], [3,7], [3,8]])
fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17, 11, -1])
optimal_path = fitted_tree.run_dfs(0, 11)
print("The Optimal Path index by DFS ", optimal_path)



Tree_edges = create_tree(9, [[0,1], [2,0], [1,3], [1,4], [1,5], [2,6], [3,7], [3,8]])
fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17, 11, -1])
optimal_path = fitted_tree.run_dfs(0, 11, 1)
print("The Optimal Path index by Depth Limited Search ", optimal_path)




fitted_tree = BFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17, 11, -1])
optimal_path = fitted_tree.run_bfs(0, 11)
print("The Optimal Path index by BFS ", optimal_path)



fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17, 11, -1])
optimal_path = fitted_tree.run_iterative_dfs(0, 11)
print("The Optimal Path index by iterative DFS ", optimal_path)

closed list [] and stack [0] and depth 99999
The Optimal Path index by DFS  [0]
closed list [] and stack [0] and depth 1
The Optimal Path index by Depth Limited Search  [0]
The Optimal Path index by BFS  [0]
closed list [] and stack [0] and depth 0
1 [0]
The Optimal Path index by iterative DFS  [0]


# TestCase 5: Solution does not exists

In [18]:
Tree_edges = create_tree(9, [[0,1], [2,0], [1,3], [1,4], [1,5], [2,6], [3,7], [3,8]])
fitted_tree = BFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17, 11, -1])
optimal_path = fitted_tree.run_bfs(0, 110)
print("The Optimal Path index by bfs", optimal_path)




fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17, 11, -1])
optimal_path = fitted_tree.run_dfs(0, 110)
print("The Optimal Path index by dfs", optimal_path)




fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17, 11, -1])
optimal_path = fitted_tree.run_dfs(0, 110, 2)
print("The Optimal Path index by depth limited search", optimal_path)






fitted_tree = DFS(Tree_edges, [11, 1, 4, 17, 8, -3, 17, 11, -1])
optimal_path = fitted_tree.run_iterative_dfs(0, 110)
print("The Optimal Path index by iterative dfs", optimal_path)

After DnQueue  []
Front:  0 Node:  1
After EnQueue  [1.]
Front:  0 Node:  2
After EnQueue  [1. 2.]
After DnQueue  [2.]
Front:  1 Node:  3
After EnQueue  [2. 3.]
Front:  1 Node:  4
After EnQueue  [2. 3. 4.]
Front:  1 Node:  5
After EnQueue  [2. 3. 4. 5.]
After DnQueue  [3. 4. 5.]
Front:  2 Node:  6
After EnQueue  [3. 4. 5. 6.]
After DnQueue  [4. 5. 6.]
Front:  3 Node:  7
After EnQueue  [4. 5. 6. 7.]
Front:  3 Node:  8
After EnQueue  [4. 5. 6. 7. 8.]
After DnQueue  [5. 6. 7. 8.]
After DnQueue  [6. 7. 8.]
After DnQueue  [7. 8.]
After DnQueue  [8.]
After DnQueue  []
No solution Exist! check the input node values
The Optimal Path index by bfs -1
closed list [] and stack [0] and depth 99999
closed list [] and stack [0. 1.] and depth 99999
closed list [] and stack [0. 1. 3.] and depth 99999
closed list [] and stack [0. 1. 3. 7.] and depth 99999
closed list [7] and stack [0. 1. 3.] and depth 99999
closed list [7] and stack [0. 1. 3. 8.] and depth 99999
closed list [7, 8] and stack [0. 1. 3.] a